In [1]:
from collections import defaultdict
import nltk
import os
import numpy as np
from nltk.corpus import machado, mac_morpho
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import string
from collections import defaultdict
from nltk.stem.snowball import PortugueseStemmer
import enchant
from gensim import corpora

In [2]:
textos = []
for p, d, f in os.walk(r'machado/machado'):
    #print( p,d,f)
    if f:
        for fileid  in f:
            if not fileid.endswith('.txt'):
                continue
            with open(os.path.join(p,fileid), encoding='iso-8859-1') as g:
                textos.append(g.read())

In [3]:
swu = stopwords.words('portuguese') + list (string.punctuation)
stemmer = PortugueseStemmer()

In [4]:
#textos = textos[:4]

In [5]:
def clean_sentence(texto : str):
    return [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
textos = [*map(clean_sentence,textos)]

In [6]:
all_words = set()
for text in textos:
    all_words |= set(text)
ordered_words = list(sorted(all_words))
index_word = dict([(word,i) for i, word in enumerate(ordered_words)])

In [7]:
matrix = [[0]*len(textos) for _ in range(len(all_words))]
for i, document in enumerate(textos):
    for j, word in enumerate(ordered_words):
        matrix[j][i] = document.count(word)
matrix = np.matrix(matrix)
binary_matrix = (matrix >= 1)*1

KeyboardInterrupt: 

## Questão 1

In [ ]:
def frequency_from(word, matrix):
    clean_word = clean_sentence(word)[0]
    if clean_word not in index_word:
        return [[0]*matrix.shape[1]]
    word_index = index_word[clean_word]
    
    line = matrix[word_index]
    return line/np.sum(line)

def binary_frequency(word):
    return frequency_from(word,binary_matrix)

def word_frequency(word):
    return frequency_from(word,matrix)

In [ ]:
binary_frequency('agora')

In [ ]:
word_frequency('agora')

In [ ]:
def tfdf(word):
    clean_word = clean_sentence(word)[0]
    if clean_word not in index_word:
        return [[0]*matrix.shape[1]]
    word_index = index_word[clean_word]
    line = matrix[word_index]
    quantity_words_per_doc = matrix.sum(axis=0)
    tf = line/quantity_words_per_doc
    
    freq_word = ((tf > 0)*1).sum()
    assert freq_word > 0
    df = np.log(matrix.shape[1]/freq_word)
    assert df >= 0
    return tf*df

In [ ]:
tfdf('agora')

## Questão 2

In [ ]:
def get_log_normalized(matrix):
    log_matrix = np.log(matrix+1)
    squared_log_matrix = np.square(log_matrix)
    sum_rows = squared_log_matrix.sum(axis=1)
    log_normalized_matrix = np.power(np.multiply(squared_log_matrix,np.power(sum_rows, -1)),0.5)
    return log_normalized_matrix

In [ ]:
log_normalized_matrix = get_log_normalized(matrix)
log_normalized_matrix

## Questão 3

In [ ]:
frequency = defaultdict(int)
for text in textos:
    for token in text:
        frequency[token] += 1
freq_texts = [[token for token in text if frequency[token] > 1]
         for text in textos]
dictionary = corpora.Dictionary(freq_texts)

In [ ]:
def to_corpus(doc):
    new_vec = dictionary.doc2bow(doc.lower().split())
    corpus = [dictionary.doc2bow(text) for text in freq_texts]
    return corpus

In [ ]:
[vet[:5] for vet in to_corpus('agora')[:5]]